In [1]:
#YOLO 사용하기위한 라이브러리
import torch
from glob import glob
import yaml
from IPython.display import Image
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import save_model, Sequential
from tensorflow.python.keras.models import load_model
import numpy as np
%cd yolov5/
import detect_simple as ds
import gc
import torch

mt_model = ''

C:\Users\smhrd\RuningMachine\yolov5


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#USE_CUDA = torch.cuda.is_available()
#print(USE_CUDA)

#device = torch.device('cuda:0' if USE_CUDA else 'cpu')
device = torch.device('cpu')
print(device)
#device

In [ ]:
def train_weight():
    now_path = %pwd
    
    if now_path == 'C:\\Users\\smhrd\\RuningMachine\\yolov5':
        !python train.py --img 240 --epochs 15 --data ./dataset/data.yaml --weights yolov5s.pt
    else:
        %cd C:\\Users\\smhrd\\RuningMachine\\yolov5
        %pwd
        !python train.py --img 240 --epochs 15 --data ./dataset/data.yaml --weights yolov5s.pt

In [ ]:
%cd yolov5/

In [ ]:
!python detect.py --weights ./runs\train\300epoch\weights\best.pt --img 240 --source ./testv2.mp4

In [ ]:
#만들어진 모델 텐서타입으로.
!python export.py --weights '.\runs\train\300epoch\weights\best.pt' --include saved_model

In [ ]:
path = 'C:/Users/smhrd/RuningMachine/yolov5/best_saved_model/'
reconstructed_model=tf.keras.models.load_model(path,compile=False)

In [ ]:
reconstructed_model.summary()

# simply detct import 하는 방식 시도

# 기존 디텍트 하는 방식에서 변경
- !python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
- #display.Image(filename='runs/detect/exp/zidane.jpg', width=600) ->>> 이부분

In [24]:
# 모션모델 분별함수
def motion_model_run(frame):
    global mt_model
    frame = cv2.resize(frame, (480,240))

    images=cv2.resize(frame,(224,224))
    images = np.array([images])
    pers=int((mt_model.predict(images)[0][1])*100)
    
    if mt_model.predict_classes(images) == 1 :
        a="CATCH"
    else :
        a=""
    
    cv2.rectangle(frame,(0,0),(90,20),(255,255,255),-1)
    cv2.putText(frame, str(a), (10,15),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,0),1,cv2.LINE_AA)
    print("정확도:",str(pers)+" %")
    cv2.putText(frame, str(pers)+" %", (55,15),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,0),1,cv2.LINE_AA)
        
    return frame , a , pers

In [25]:
from flask import Flask,render_template, request, Response, redirect, url_for, abort, stream_with_context, send_file, jsonify
from werkzeug.utils import secure_filename
import cv2
%cd yolov5/
import detect_simple as ds

app = Flask(__name__)
  
def gen_frames(file):
    size = (480,240)
    #업로드 한 영상을 파일 로드 하는 부분
    cap = cv2.VideoCapture(os.path.join('./',file))
    while True:
 
        # # Capture frame-by-frame. Return boolean(True=frame read correctly. )
        success, frame = cap.read()  # read the camera frame
        if not success:
            break
        else:
            #범용성 있게 쓰겠금 변수선언
            resultImage = ''
            #모션분류 (상민씨)
            mt_frame,mt_anwser,mt_per = motion_model_run(frame)
            #Yolov5 객체탐지 (서인원,김정식)
            yl_img , yl_result = ds.detect(mt_frame)
            
            #원본이미지의 각 결과값 (모션분류 결과 이미지) + 객체탐지 ( 결과 이미지 )
            
            #cv2.rectangle(yl_img,(0,0),(90,20),(255,255,255),-1)
            #cv2.putText(yl_img, str(mt_anwser), (10,15),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,0),1,cv2.LINE_AA)
            #print("정확도:",str(mt_per)+" %")
            #cv2.putText(yl_img, str(mt_per)+" %", (55,15),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,0),1,cv2.LINE_AA)
            
            if 'CATCH' in mt_anwser:
                #print("캐치 하였음!")
                #상우 로직 작동 태울 것.
                
                
                
                #이미지 좌우 합치기
                front_result = cv2.resize(yl_img,size) #front : 모션분류 , 객체탐지 처리완료된 이미지
                back_result = cv2.resize(mt_frame,size) #back : 상황 캐치  , 얼굴 탐지 
                #좌우 합치기
                resultImage = np.hstack((front_result,back_result))
            # 절도 아닌 이상
            else :
                #print("절도 아닌 상황!")
                resultImage = yl_img
            # q를 누르면 종료
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break 

            #GPU 사용하기위해서 메모리 제거
            gc.collect()
            torch.cuda.empty_cache()
            
            #리턴받는것
            ret, buffer = cv2.imencode('.jpg', resultImage)
            #객체인식관련 코드 
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # concat frame one by one and show result
 
 
@app.route('/video_feed', methods=["GET"])
def video_feed():
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/', methods=["GET",'POST'])
def render_file():
    return render_template("upload.html") # 업로드용 페이지 html 보여주기

#파일 업로드 처리
@app.route("/fileUpload", methods=['GET',"POST"])
def file_upload():
    if request.method=='POST':
        f = request.files["file"] # html <input name= "file">        
        
        #업로드 할때 파일 이름 출력
        FILE_NAME = f.filename 
        f.save(secure_filename(f.filename)) # test_park.mp4
        
        return Response(gen_frames(f.filename),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
@app.route("/view") 
def view(): 
    return render_template("view.html", image_file=f"./static/{FILE_NAME}.jpg") # html페이지를 보내기

@app.route("/view2")
def view2():
    return send_file(f"./static/{FILE_NAME}.jpg") # 이미지 파일만 보내기

if __name__ == '__main__':
    global mt_model
    mt_model = load_model("./test_set/modelAbur5.npz.h5")
    app.run(host='121.147.0.176',port=2000)
    app.debug=True

[WinError 2] 지정된 파일을 찾을 수 없습니다: 'yolov5/'
C:\Users\smhrd\RuningMachine\yolov5
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://121.147.0.176:2000/ (Press CTRL+C to quit)
121.147.0.176 - - [07/Feb/2022 11:01:44] "GET / HTTP/1.1" 200 -
YOLOv5  v6.0-193-gdb1f83b torch 1.10.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 


정확도: 0 %
사용 모델: runs/train/300epoch/weights/best.pt
device: cuda:0


Model Summary: 213 layers, 7371523 parameters, 0 gradients, 17.0 GFLOPs
121.147.0.176 - - [07/Feb/2022 11:01:47] "POST /fileUpload HTTP/1.1" 200 -


객체명:snack_pepero, 확률:0.42%
Inferencing and Processing Done. (0.033s)
정확도: 0 %
사용 모델: runs/train/300epoch/weights/best.pt


...

객체명:coffee_, 확률:0.56%
Inferencing and Processing Done. (0.014s)
정확도: 84 %


YOLOv5  v6.0-193-gdb1f83b torch 1.10.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 


사용 모델: runs/train/300epoch/weights/best.pt
device: cuda:0


Model Summary: 213 layers, 7371523 parameters, 0 gradients, 17.0 GFLOPs


객체명:coffee_, 확률:0.56%
Inferencing and Processing Done. (0.013s)


In [ ]:
!python detect.py --weights ./runs/train/300epoch/weights/best.pt --img 240 --conf 0.25 --source ./test_data//
#가장 최근 만들어진 폴더찾고 그 경로로 설정해주기
#source 집어넣을때 이름은 있음. exp...만드는데.. 아 +1 시켜가지고 가져오기.
dir_list = os.listdir('./test_data/')
result_list = os.listdir('./runs/detect/')
count = len(result_list)

if count == 1 : 
    count = ''
    
print(count)

#for filename in dir_list:
#    target = './runs/detect/'+f'exp{str(count)}/'+filename
#    print('타겟스'+target)
#    display(Image(filename=target,width=240,height=240))

In [ ]:
# 모델 저장  처리단계
from models.yolo import Model
device = torch.device("cpu")
#device = torch.device("cuda")
yaml_path = './models/yolov5s.yaml'
new_weights='./runs/train/300epoch/weights/best.pt'
model= Model(yaml_path).to(device)
trained_model = torch.load(new_weights,map_location=device)['model'].to(device)

In [ ]:
#실제 모델 저장하는 부분
torch.save(trained_model,'best_1.h5')

In [ ]:
%cd yolov5/

In [ ]:
#모델 로드 하는 부분
loaded_model=torch.load('./best_1.pt')

In [ ]:
loaded_model.load_state_dict

In [ ]:
loaded_model.names

In [ ]:
from PIL import Image
import torchvision
from torchvision import transforms
import numpy as np
#from utils.datasets import letterbox

In [ ]:
loaded_model=torch.load('./best_1.h5')

#이미지 경로
imgpath = './test_data/amond.jpg'
#img = '../zidane.jpg'
#img=cv2.imread(imgpath)
img = Image.open(imgpath)
tf = transforms.ToTensor()
img_t = tf(img)
print(type(img))

In [ ]:
img_t = img_t.resize_(32, 3, 6, 6)
print(img_t)

In [ ]:
results = loaded_model(img_t)

In [ ]:
for result in results.crop():
    print(result['label'].uni)

# 재학습 하는 코딩 ( 기존 가중치 파일 ++누적 하고자할때 쓰는 코드)
#### 나중에 함수화 진행 예정 
- !python val.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65 --half

In [ ]:
#모델 샘플 코드
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom
# Images
img = 'https://ultralytics.com/images/zidane.jpg'  # or file, Path, PIL, OpenCV, numpy, list
# Inference
#print(model.load_state_dict)
results = model(img)

# Results
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

In [ ]:
for result in results.crop():
    print(result['label'])